# ImageNet预训练图像分类模型预测视频文件

使用 ImageNet 预训练图像分类模型，对视频文件执行预测。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

## 设置中文字体

In [1]:
# # windows操作系统
# plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签 
# plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号

In [2]:
# Mac操作系统，参考 https://www.ngui.cc/51cto/show-727683.html
# 下载 simhei.ttf 字体文件
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/SimHei.ttf

In [1]:
# Linux操作系统，例如 云GPU平台：https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1
# 如果遇到 SSL 相关报错，重新运行本代码块即可
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/SimHei.ttf -O /environment/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf
!rm -rf /home/featurize/.cache/matplotlib

import matplotlib
matplotlib.rc("font",family='SimHei') # 中文字体


--2022-07-24 22:32:47--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/SimHei.ttf
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 10050868 (9.6M) [application/x-font-ttf]
Saving to: ‘/environment/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf’

/environment/minico 100%[===================>]   9.58M  21.8MB/s    in 0.4s    

2022-07-24 22:32:53 (21.8 MB/s) - ‘/environment/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/SimHei.ttf’ saved [10050868/10050868]



## 导入中文字体

In [14]:
from PIL import ImageFont, ImageDraw
# 导入中文字体，指定字号
font = ImageFont.truetype('SimHei.ttf', 32)

## 导入工具包

In [12]:
import os
import cv2 # opencv-python
from PIL import Image, ImageFont, ImageDraw
from tqdm import tqdm # 进度条

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
from torchvision import models

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号
plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签
import gc

import mmcv
import tempfile
import shutil
import time

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cuda:0


In [2]:
# 后端绘图，不显示，只保存
import matplotlib
matplotlib.use('Agg')

## 载入模型

In [3]:
# 载入预训练图像分类模型
model = models.resnet18(pretrained=True)
model = model.eval()
model = model.to(device)

In [4]:
# 载入ImageNet 1000图像分类标签
df = pd.read_csv('imagenet_class_index.csv')

In [5]:
idx_to_labels = {}
for idx, row in df.iterrows():
    idx_to_labels[row['ID']] = [row['wordnet'], row['Chinese']]

In [6]:
# idx_to_labels

## 图像预处理

In [7]:
from torchvision import transforms

# 测试集图像预处理-RCTN：缩放裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 视频预测

### 输入输出视频路径

In [8]:
input_video = 'test_img/video_3.mp4'

### 可视化方案一：原始图像+预测结果文字

In [15]:
def classify_single_img(img_path, n=5):
    '''
    输入图像文件路径，输出图像分类前 n 个预测结果，保存输出图像
    '''
    img_bgr = cv2.imread(img_path)
    img_pil = Image.open(img_path)
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    # 取置信度最大的 n 个结果
    top_n = torch.topk(pred_softmax, n) 
    confs = top_n[0].cpu().detach().numpy().squeeze()
    pred_ids = top_n[1].cpu().detach().numpy().squeeze()

    pred_df = pd.DataFrame() # 预测结果表格
    for i in range(n):
        class_name = idx_to_labels[pred_ids[i]][1] # 获取类别名称
        label_idx = int(pred_ids[i]) # 获取类别号
        wordnet = idx_to_labels[pred_ids[i]][0] # 获取 WordNet
        confidence = confs[i] * 100 # 获取置信度
        pred_df = pred_df.append({'Class':class_name, 'Class_ID':label_idx, 'Confidence(%)':confidence, 'WordNet':wordnet}, ignore_index=True) # 预测结果表格添加一行

    top_n = torch.topk(pred_softmax, n) 
    confs = top_n[0].cpu().detach().numpy().squeeze()
    pred_ids = top_n[1].cpu().detach().numpy().squeeze()
    pred_df = pd.DataFrame() # 预测结果表格
    for i in range(n):
        class_name = idx_to_labels[pred_ids[i]][1] # 获取类别名称
        label_idx = int(pred_ids[i]) # 获取类别号
        wordnet = idx_to_labels[pred_ids[i]][0] # 获取 WordNet
        confidence = confs[i] * 100 # 获取置信度
        pred_df = pred_df.append({'Class':class_name, 'Class_ID':label_idx, 'Confidence(%)':confidence, 'WordNet':wordnet}, ignore_index=True) # 预测结果表格添加一行
    
    draw = ImageDraw.Draw(img_pil) 
    # 在图像上写字
    for i in range(len(confs)):
        pred_class = idx_to_labels[pred_ids[i]][1]
        text = '{:<15} {:>.3f}'.format(pred_class, confs[i])
        # 文字坐标，中文字符串，字体，rgba颜色
        draw.text((50, 100 + 50 * i), text, font=font, fill=(255, 0, 0, 1))
    img_pred_1 = np.array(img_pil) # PIL 转 array
    
    # 保存图像
    img_pred_rgb = cv2.cvtColor(img_pred_1, cv2.COLOR_RGB2BGR) # RGB转BGR
    cv2.imwrite('temp/视频单帧预测结果.jpg', img_pred_rgb)
    

In [16]:
# 创建临时文件夹，存放每帧结果
temp_out_dir = time.strftime('%Y%m%d%H%M%S')
os.mkdir(temp_out_dir)
print('创建文件夹 {} 用于存放每帧预测结果'.format(temp_out_dir))

创建文件夹 20220803194809 用于存放每帧预测结果


In [17]:
# 读入待预测视频
imgs = mmcv.VideoReader(input_video)

prog_bar = mmcv.ProgressBar(len(imgs))

# 对视频逐帧处理
for frame_id, img in enumerate(imgs):
    
    cv2.imwrite('temp/视频单帧画面.jpg', img)
    classify_single_img('temp/视频单帧画面.jpg', n=5)
    img = cv2.imread('temp/视频单帧预测结果.jpg')

    # 将处理后的该帧画面图像文件，保存至 /tmp 目录下
    cv2.imwrite(f'{temp_out_dir}/{frame_id:06d}.jpg', img)
    
    prog_bar.update() # 更新进度条

# 把每一帧串成视频文件
mmcv.frames2video(temp_out_dir, 'output/output_pred.mp4', fps=imgs.fps, fourcc='mp4v')

shutil.rmtree(temp_out_dir) # 删除存放每帧画面的临时文件夹

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 132/132, 24.1 task/s, elapsed: 5s, ETA:     0s[                                                  ] 0/132, elapsed: 0s, ETA:


### 可视化方案二：原始图像+预测结果文字+各类别置信度柱状图

In [18]:
def classify_single_img(img_path, n=5):
    '''
    输入图像文件路径，输出图像分类前 n 个预测结果，保存输出图像
    '''
    img_bgr = cv2.imread(img_path)
    img_pil = Image.open(img_path)
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    # 取置信度最大的 n 个结果
    top_n = torch.topk(pred_softmax, n) 
    confs = top_n[0].cpu().detach().numpy().squeeze()
    pred_ids = top_n[1].cpu().detach().numpy().squeeze()

    pred_df = pd.DataFrame() # 预测结果表格
    for i in range(n):
        class_name = idx_to_labels[pred_ids[i]][1] # 获取类别名称
        label_idx = int(pred_ids[i]) # 获取类别号
        wordnet = idx_to_labels[pred_ids[i]][0] # 获取 WordNet
        confidence = confs[i] * 100 # 获取置信度
        pred_df = pred_df.append({'Class':class_name, 'Class_ID':label_idx, 'Confidence(%)':confidence, 'WordNet':wordnet}, ignore_index=True) # 预测结果表格添加一行

    top_n = torch.topk(pred_softmax, n) 
    confs = top_n[0].cpu().detach().numpy().squeeze()
    pred_ids = top_n[1].cpu().detach().numpy().squeeze()
    pred_df = pd.DataFrame() # 预测结果表格
    for i in range(n):
        class_name = idx_to_labels[pred_ids[i]][1] # 获取类别名称
        label_idx = int(pred_ids[i]) # 获取类别号
        wordnet = idx_to_labels[pred_ids[i]][0] # 获取 WordNet
        confidence = confs[i] * 100 # 获取置信度
        pred_df = pred_df.append({'Class':class_name, 'Class_ID':label_idx, 'Confidence(%)':confidence, 'WordNet':wordnet}, ignore_index=True) # 预测结果表格添加一行
    
    draw = ImageDraw.Draw(img_pil) 
    # 在图像上写字
    for i in range(len(confs)):
        pred_class = idx_to_labels[pred_ids[i]][1]
        text = '{:<15} {:>.3f}'.format(pred_class, confs[i])
        # 文字坐标，中文字符串，字体，rgba颜色
        draw.text((50, 100 + 50 * i), text, font=font, fill=(255, 0, 0, 1))
    img_pred_1 = np.array(img_pil) # PIL 转 array
    
    # 保存图像
    img_pred_rgb = cv2.cvtColor(img_pred_1, cv2.COLOR_RGB2BGR) # RGB转BGR
    cv2.imwrite('output/pred_result_1.jpg', img_pred_rgb)
    
    ## 图像和柱状图一起显示
    # 载入原始图像
    img = cv2.imread('temp/pred_result_1.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # BGR 转 RGB
    fig = plt.figure(figsize=(18,6))
    # 绘制左图-视频图
    ax1 = plt.subplot(1,2,1)
    ax1.imshow(img_pred_1)
    ax1.axis('off')
    # 绘制右图-柱状图
    ax2 = plt.subplot(1,2,2)
    x = df['ID']
    y = pred_softmax.cpu().detach().numpy()[0]
    ax2.bar(x, y, alpha=0.5, width=0.3, color='yellow', edgecolor='red', lw=3)
    plt.xlabel('类别', fontsize=20)
    plt.ylabel('置信度', fontsize=20)
    ax2.tick_params(labelsize=16) # 坐标文字大小
    plt.ylim([0, 1.0]) # y轴取值范围
    plt.xlabel('类别',fontsize=25)
    plt.ylabel('置信度',fontsize=25)
    plt.title('图像分类预测结果', fontsize=30)
    plt.tight_layout()
    fig.savefig('temp/视频单帧预测结果.jpg')
    # 释放内存
    fig.clf()
    plt.close()
    gc.collect()

In [19]:
# 创建临时文件夹，存放每帧结果
temp_out_dir = time.strftime('%Y%m%d%H%M%S')
os.mkdir(temp_out_dir)
print('创建文件夹 {} 用于存放每帧预测结果'.format(temp_out_dir))

创建文件夹 20220803195013 用于存放每帧预测结果


In [20]:
# 读入待预测视频
imgs = mmcv.VideoReader(input_video)

prog_bar = mmcv.ProgressBar(len(imgs))

# 对视频逐帧处理
for frame_id, img in enumerate(imgs):
    
    cv2.imwrite('temp/视频单帧画面.jpg', img)
    classify_single_img('temp/视频单帧画面.jpg', n=5)
    img = cv2.imread('temp/视频单帧预测结果.jpg')

    # 将处理后的该帧画面图像文件，保存至 /tmp 目录下
    cv2.imwrite(f'{temp_out_dir}/{frame_id:06d}.jpg', img)
    
    prog_bar.update() # 更新进度条

# 把每一帧串成视频文件
mmcv.frames2video(temp_out_dir, 'output/output_bar.mp4', fps=imgs.fps, fourcc='mp4v')

shutil.rmtree(temp_out_dir) # 删除存放每帧画面的临时文件夹

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 132/132, 93.6 task/s, elapsed: 1s, ETA:     0s[                                                  ] 0/132, elapsed: 0s, ETA:
